# Hierarchy

Python code can be ordered using the following hierarchy.
Note that some hierarchy levels are not applicable for Jupyter Notebooks but only for "normal" Python code.

## Packages [optional]
A directory conaining .py files (i.e. Modules) and/ or subpackages. A directory becomes a Python Package if the file 

    __init__.py 
    
(which is usually empty) is present.

## Modules
A file containing Python code. It may be top level or inside a Package. A module may contain functions, classes, variables (usually constants) and/ or code.
    
Code inside a Module is automatically executed if the module is imported. Code which should be only executed if the module itself is executed (not by imports from other modules) is protected using the following check:
    
    if __name__ == '__main__':
        code
        
Note that it is perfectly fine in Python to have constants and functions directly in Modules, outside classes (unlike e.g. in Java, where everything must be inside a class). The module-level namespaces avoid naming conflicts.

# Imports

In [11]:
%cat ../example_files/my_module.py # example module to be imported

# example file for module imports

__version__ = '0.0.1'

my_secret = 'you_will_never_guess'
my_list = [1, 2, 3, 4]
my_tuple = ('a', 'b', 'c')

def double_me(x):
    return 2*x

class Multiplier:
    def __init__(self, i):
        self.i = i
    def get_result(self, x):
        return self.i*x

mult2 = Multiplier(2)
mult3 = Multiplier(3)

print('my_module imported')

if __name__ == '__main__':
    # this code is not executed when importing this module, but only if the module is executed
    print('executing main block')


## Pythonpath

In [12]:
import sys
import os

In [13]:
new_path = os.path.abspath('../example_files')
if new_path not in sys.path:
    sys.path.append(new_path)

Python searches for modules in import statements in the following places:
* Current directory
* Python builtins
* Directories in Pythonpath

The code above checks if the given directory is included in Pythonpath and adds it if not.

This pattern is also useful e.g. for unit tests if the test cases are kept in a directory different from the code.

## Import Specific Objects

In [14]:
from my_module import double_me
from my_module import my_list as my_constants # rename object with importings

The 

    from module import object [as alias]
    
pattern imports specific objects of the module in local namespace.

Note that the print statement at the bottom of the module is executed, although it is not in one of the imported objects. Python runs the complete module script the first time it is used in any import.

In [15]:
double_me(42)

84

In [16]:
my_constants

[1, 2, 3, 4]

#### Antipatterns

In [17]:
my_constants.append(5)
my_constants

[1, 2, 3, 4, 5]

Extremely dangerous: changing (mutable) data in an other module. All imports referring to this data will be affected.

In [18]:
my_tuple = ('this', 'is', 'very', 'important')
my_tuple

('this', 'is', 'very', 'important')

In [19]:
from my_module import * # Imports all members of the module into local namespace

In [20]:
my_tuple

('a', 'b', 'c')

The usage of the pattern

    from module import *

has the following drawbacks and is therefore not recommended:
* There could be naming conflicts between local and imported objects
* There is no information or control in the module where the import statement has been executed which object originates from which module.

## Import Whole Module as Separate Namespace

In [21]:
import my_module

This is the recommended method to import modules, especially if multiple objects of the module are required. If the number of imported objects is small, the *from module import object* syntax is also fine.

The code in the imported module is only executed once during the first import. Thus the print statement is not shown again.

In [22]:
my_module.double_me(42)

84

In [23]:
mult7 = my_module.Multiplier(7)
mult7.get_result(7)

49

In [24]:
my_module.mult2.get_result(3)

6

In [25]:
my_module.my_list # note the effect of my_constants.append(5)

[1, 2, 3, 4, 5]

In [26]:
my_module.my_list is my_constants # both are pointers to the same list object

True

Note that the modification of the imported list changes this list anywhere it is imported (actually, all imports refer to the same object). 
Therefore it is very dangerous to import and modify mutable data structures from other modules.

## Import from Packages

In [27]:
import mypackage # note that this does not import submodules in this package

In [28]:
try:
    mypackage.my_module2.triple_me(3)
except AttributeError as e:
    print(e)

module 'mypackage' has no attribute 'my_module2'


In [29]:
import mypackage.my_module2 # explicit import of modules in package into package namespace

In [30]:
mypackage.my_module2.triple_me(3)

9

In [31]:
from mypackage import my_module2 # import submodules into current namespace

In [32]:
my_module2.triple_me(3)

9

In [33]:
from mypackage.my_module2 import triple_me as tripler
# import objects from submodules into current namespace

In [34]:
tripler(4)

12

In [35]:
from mypackage import * # imports only content/imports of __init__.py and modules
# listed in the __all__ variable in the init file into current namespace

In [41]:
%cat ../example_files/mypackage/__init__.py # note that my_module4 is included here, but not my_module3

__all__ = ['my_module4']


In [37]:
try:
    my_module3.square_me(4) # not imported with from package import *
except NameError as e:
    print(e)

name 'my_module3' is not defined


In [38]:
my_module4.divide_me_by_2(8) # imported with from package import * because module name
# is in __all__ of __init__.py

4.0

# Main Block

In some cases, the same Python module id used both in imports and executed directly.
In this case, a main block can be defined which is not executed during import. There is no pre-defined method to do this in Python (like the main class in Java), but the following code pattern is used:

    if __name__ == '__main__':
        do_something
        


In [39]:
import my_module # note that the print command inside the main block is not executed 

In [40]:
exec(open('../example_files/my_module.py').read()) # antipattern: never use this as part of a program workflow
# use import instead

my_module imported
executing main block
